#Rock-Paper-Scissors

## IMPORTING MODULES

In [8]:
import os
import pickle
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Running on", device)


Running on cpu


## Load and Inspect Data


In [9]:
with open('train.pkl', 'rb') as f:
    train_data = pickle.load(f)
imgs1 = np.stack([t[0] for t in train_data])   # (N,24,24)
imgs2 = np.stack([t[1] for t in train_data])
labels = np.array([t[2] for t in train_data])  # +1 / -1
train_ids = np.array([t[3] for t in train_data])
print("Train:", imgs1.shape, "Label counts:", np.unique(labels, return_counts=True))

with open('test.pkl', 'rb') as f:
    test_data = pickle.load(f)
test_imgs1 = np.stack([t[0] for t in test_data])
test_imgs2 = np.stack([t[1] for t in test_data])
test_ids   = np.array([t[3] for t in test_data])
print("Test:", test_imgs1.shape, "# IDs:", len(test_ids))

# Combine into shape (N,2,24,24)
X = np.stack([imgs1, imgs2], axis=1)      # train pairs
X_test = np.stack([test_imgs1, test_imgs2], axis=1)


# ========================================
# 3. Train/Validation Split
# ========================================
X_train, X_val, y_train, y_val = train_test_split(
    X, labels, test_size=0.2, stratify=labels, random_state=42
)
print("Train split:", X_train.shape, "Val split:", X_val.shape)


# ========================================
# 4. Simple CNN Definition
# ========================================
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # input: (batch,2,24,24)
        self.conv = nn.Sequential(
            nn.Conv2d(2, 16, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),   # →16×12×12
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),  # →32×6×6
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*6*6, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        x = self.conv(x)
        return self.fc(x).squeeze(-1)  # logits


# ========================================
# 5. PyTorch Dataset & DataLoader
# ========================================
class RPSDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img = torch.from_numpy(self.X[idx])            # (2,24,24)
        if self.y is None:
            return img
        lbl = 1.0 if self.y[idx] > 0 else 0.0          # +1→1, -1→0
        return img, torch.tensor(lbl, dtype=torch.float32)


# ========================================
# 6. CNN Training Function
# ========================================
def train_cnn(model, train_loader, val_loader, epochs=10, lr=1e-3):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    best_acc, best_wts = 0.0, None

    for ep in range(1, epochs+1):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            loss = criterion(logits, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        all_preds, all_lbls = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                logits = model(xb).cpu().numpy()
                probs = 1 / (1 + np.exp(-logits))
                all_preds.extend(probs)
                all_lbls.extend(yb.numpy())
        bin_preds = [1 if p>0.5 else 0 for p in all_preds]
        acc = accuracy_score(all_lbls, bin_preds)
        print(f"Epoch {ep}/{epochs} — val_acc: {acc:.4f}")
        if acc > best_acc:
            best_acc = acc
            best_wts = {k:v.cpu() for k,v in model.state_dict().items()}

    model.load_state_dict(best_wts)
    return model


# ========================================
# 7. Train Two CNNs (Different Seeds)
# ========================================
bs = 64
train_ds = RPSDataset(X_train, y_train)
val_ds   = RPSDataset(X_val,   y_val)
train_ld = DataLoader(train_ds, batch_size=bs, shuffle=True)
val_ld   = DataLoader(val_ds,   batch_size=bs)

cnn_models = []
for seed in [0, 1]:
    torch.manual_seed(seed)
    np.random.seed(seed)
    print(f"\n> Training CNN with seed={seed}")
    net = SimpleCNN()
    net = train_cnn(net, train_ld, val_ld, epochs=8, lr=1e-3)
    cnn_models.append(net)


# ========================================
# 8. Extract CNN Probabilities
# ========================================
def get_probs(model, X_arr):
    ds = RPSDataset(X_arr, y=None)
    ld = DataLoader(ds, batch_size=bs)
    model.eval()
    probs = []
    with torch.no_grad():
        for xb in ld:
            xb = xb.to(device)
            logits = model(xb).cpu().numpy()
            probs.extend(1 / (1 + np.exp(-logits)))
    return np.array(probs)

train_cnn_feats = np.vstack([get_probs(m, X_train) for m in cnn_models]).T
val_cnn_feats   = np.vstack([get_probs(m, X_val)   for m in cnn_models]).T
test_cnn_feats  = np.vstack([get_probs(m, X_test)  for m in cnn_models]).T
print("CNN feature shapes:", train_cnn_feats.shape, val_cnn_feats.shape)


# ========================================
# 9. Boosted Trees on Raw-Pixel Differences
# ========================================
def make_flat(X_arr):
    dif = X_arr[:,0] - X_arr[:,1]      # (N,24,24)
    return dif.reshape(len(dif), -1)   # (N,576)

Xtr_flat = make_flat(X_train)
Xvl_flat = make_flat(X_val)
Xte_flat = make_flat(X_test)

gbm = GradientBoostingClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)
gbm.fit(Xtr_flat, (y_train>0).astype(int))
boost_tr_p = gbm.predict_proba(Xtr_flat)[:,1]
boost_vl_p = gbm.predict_proba(Xvl_flat)[:,1]
boost_te_p = gbm.predict_proba(Xte_flat)[:,1]

print("GBM val acc:", 
      accuracy_score((y_val>0).astype(int), boost_vl_p>0.5))


# ========================================
# 10. Stacked Meta-Learner
# ========================================
stack_tr = np.column_stack([train_cnn_feats, boost_tr_p])
stack_vl = np.column_stack([val_cnn_feats,   boost_vl_p])
stack_te = np.column_stack([test_cnn_feats,  boost_te_p])

meta = LogisticRegression()
meta.fit(stack_tr, (y_train>0).astype(int))

vl_pred = meta.predict(stack_vl)
print("Meta val acc:", accuracy_score((y_val>0).astype(int), vl_pred))


# ========================================
# 11. Final Test Predictions & Submission
# ========================================
te_pred = meta.predict(stack_te)
te_label = np.where(te_pred>0, 1, -1)

submission = pd.DataFrame({
    'ID': test_ids,
    'Predicted': te_label
})
submission.to_csv('submission.csv', index=False)
print("Saved ▶ submission.csv")

ModuleNotFoundError: No module named 'numpy._core.numeric'